In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV

In [7]:
df = pd.read_csv('./datasets/dataset.csv', sep=',')
df = df[['compute_version','registers','smem','cmem','num_of_cores','L2','bandwith','theoretical_flops','AppId','occupancy','input_size','duration','block_x']]
df = df.select_dtypes(exclude=['object'])
df = df.sort_values(by='AppId', ascending=True)

In [11]:

regressor = Ridge()  # Alterado para usar Ridge Regression

for idApp in range(1, 10):
    X_train = df[df['AppId'] != idApp]
    Y_train = df[df['AppId'] != idApp]['duration']
    X_test = df[df['AppId'] == idApp]
    Y_test = df[df['AppId'] == idApp]['duration']

    yTrain = np.log(Y_train + 0.0001)
    yTest = np.log(Y_test + 0.0001)
  
    # Normalize seus recursos
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Aplique a busca por grid para ajustar o hiperparâmetro alpha da regularização Ridge
    param_grid = {'alpha': [0.1, 1.0, 10.0]}  # Valores para alpha que você deseja testar
    grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train_scaled, Y_train)

    best_alpha = grid_search.best_params_['alpha']
    ridge_regressor_best = Ridge(alpha=best_alpha)
    ridge_regressor_best.fit(X_train_scaled, Y_train)

    # Faça previsões com o modelo ajustado
    y_pred_best = ridge_regressor_best.predict(X_test_scaled)

    mape = round(mean_absolute_percentage_error(Y_test, y_pred_best) * 100, 2)
    print('Application:', idApp, 'Percentage:', mape, '%')


Application: 1 Percentage: 0.2 %
Application: 2 Percentage: 1.98 %
Application: 3 Percentage: 0.86 %
Application: 4 Percentage: 1.53 %
Application: 5 Percentage: 20.58 %
Application: 6 Percentage: 13.07 %
Application: 7 Percentage: 5.48 %
Application: 8 Percentage: 25.78 %
Application: 9 Percentage: 13.6 %
